# Project Sentinel Spectral Tiles

**Timm Nawrocki**  
Alaska Center for Conservation Science  
2019-04-01

In [1]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Format Sentinel Spectral Features
# Author: Timm Nawrocki
# Created on: 2019-04-01
# Usage: Must be executed as a Jupyter Notebook in an ArcGIS Pro Python 3 installation.
# Description: "Format Sentinel Spectral Features" projects spectral data from Sentinel 2 with bilinear interpolation and extracts to the area of interest with standardized grid and cell size.

## 1. Initialize Environment

In [10]:
# Import packages
import arcpy
from arcpy.sa import *
import datetime
import os
import pandas as pd
import time

# Set overwrite option
arcpy.env.overwriteOutput = True

# Set root directory
drive = 'E:/'
root_directory = os.path.join(drive, 'ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS')

# Set arcpy working environment
arcpy.env.workspace = os.path.join(root_directory, 'BristolBay_Vegetation.gdb')

# Define script paths
script_directory = os.path.join(drive, 'ACCS_Work/Repositories/southwest-alaska-moose/modules')
arcpy_geoprocessing_script = os.path.join(script_directory, 'arcpy_geoprocessing.py')

# Define input datasets
area_of_interest = os.path.join(root_directory, 'Data_Input/area_of_interest/area_of_interest.tif')
nuyakuk_subwatersheds = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_NuyakukNushagak_Subwatersheds')
aleknagik_subwatersheds = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_AleknagikWood_Subwatersheds')

# Define output datasets
mask_raster = os.path.join(root_directory, 'Data_Input/area_of_interest/NuyakukAleknagik_Mask.tif')

# Define tiles and store in dataframe for searchable access
tiles_raw = os.path.join(root_directory, 'Data_Input/source_data/imagery/sentinel2/tiles_raw')
tile_list = []
for file in os.listdir(tiles_raw):
    if file.endswith('tif'):
        tile_list.append(os.path.join(tiles_raw, file))
print('{0} raster tiles will be processed...'.format(len(tile_list)))

# Define directory for formatted tiles
tiles_project = os.path.join(root_directory, 'Data_Input/source_data/imagery/sentinel2/tiles_project')

186 raster tiles will be processed...


In [3]:
# Import and execute arcpy_geoprocessing.py
try:
    exec(open(arcpy_geoprocessing_script).read())
except:
    print("Error loading arcpy_geoprocessing.py; ensure that script directory is correct:")
    print(script_directory)
    quit()

## 2. Prepare Mask Layer

In [6]:
# Define a function to merge and dissolve input features into mask feature
def prepare_mask(**kwargs):
    """
    Description: creates a mask layer from feature classes to downscale the analysis from the entire area of interest
    Inputs: 'input_array' -- an array containing the area of interest raster and the input features classes
            'output_array' -- an array of one output name for the mask feature
    """
    # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    input_array = kwargs['input_array']
    area_of_interest = input_array.pop(0)
    output_mask = kwargs['output_array'][0]
    # Define intermediate datasets
    aoi_polygon = os.path.join(arcpy.env.workspace, 'aoi_polygon')
    merge_feature = os.path.join(arcpy.env.workspace, 'merge_feature')
    dissolve_feature = os.path.join(arcpy.env.workspace, 'dissolve_feature')
    buffer_feature = os.path.join(arcpy.env.workspace, 'buffer_feature')
    clip_feature = os.path.join(arcpy.env.workspace, 'clip_feature')
    # Convert area of interest to polygon
    print('Converting area of interest to polygon...')
    arcpy.RasterToPolygon_conversion(area_of_interest, aoi_polygon, 'NO_SIMPLIFY', 'VALUE', 'MULTIPLE_OUTER_PART')
    # Merge features into single feature class and dissolve
    print('Merging features into initial mask...')
    arcpy.Merge_management(input_array, merge_feature)
    print('Dissolving mask features...')
    arcpy.Dissolve_management(merge_feature, dissolve_feature, '', '', 'MULTI_PART', 'DISSOLVE_LINES')
    # Buffer mask feature by 100 meters
    print('Buffering mask by 100 meters...')
    arcpy.Buffer_analysis(dissolve_feature, buffer_feature, '100 Meters', 'FULL', 'ROUND', 'ALL', '', 'PLANAR')
    # Clip feature to area of interest polygon
    print('Clipping mask to area of interest...')
    arcpy.Clip_analysis(buffer_feature, aoi_polygon, clip_feature)
    # Set the snap raster and cell size environments to match the area of interest
    arcpy.env.snapRaster = area_of_interest
    arcpy.env.cellSize = area_of_interest
    # Convert the mask to raster
    print('Converting mask to raster...')
    arcpy.PolygonToRaster_conversion(clip_feature, 'OBJECTID', output_mask, 'CELL_CENTER', '', 10)
    # Delete intermediate datasets
    arcpy.Delete_management(aoi_polygon)
    arcpy.Delete_management(merge_feature)
    arcpy.Delete_management(dissolve_feature)
    arcpy.Delete_management(buffer_feature)
    arcpy.Delete_management(clip_feature)
    # End timing function execution and calculate elapsed time
    end = time.time()
    elapsed = int(end - start)
    success_time = datetime.datetime.now()
    # Report process success
    print('Successfully created mask feature...')
    print('----------')
    out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                datetime.timedelta(seconds=elapsed))
    return out_process

In [7]:
# Define key word values for the mask raster
mask_raster_inputs = [area_of_interest, nuyakuk_subwatersheds, aleknagik_subwatersheds]
mask_raster_outputs = [mask_raster]

# Create key word arguments
mask_raster_kwargs = {'input_array' : mask_raster_inputs,
                      'output_array' : mask_raster_outputs
                     }

# Create the mask feature
arcpy_geoprocessing(prepare_mask, **mask_raster_kwargs)

Converting area of interest to polygon...
Merging features into initial mask...
Dissolving mask features...
Buffering mask by 100 meters...
Clipping mask to area of interest...
Converting mask to raster...
Successfully created mask feature...
----------
Succeeded at 2019-04-06 17:18 (Elapsed time: 0:01:11)


## 3. Extract and Project Spectral Tiles

In [11]:
# Define a function to extract and project image tiles
def project_tile(**kwargs):
    """
    Description: extracts image tiles to mask feature and projects to NAD 1983 Alaska Albers using the bilinear interpolation
    Inputs: 'input_array' -- an array containing the input rasters with the first raster as the mask raster and the second raster as the image tile
            'output_array' -- an array of one output name
    """
    # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    mask_raster = kwargs['input_array'][0]
    input_raster = kwargs['input_array'][1]
    output_raster = kwargs['output_array'][0]
    output_path = os.path.split(output_raster)[0]
    # Project the tile if the output does not already exist
    if arcpy.Exists(output_raster) == False:
        print('----------')
        print('Processing raster tile {0}...'.format(os.path.split(output_raster)[1]))
        # Set the snap raster and cell size environments to match the input raster
        arcpy.env.snapRaster = input_raster
        arcpy.env.cellSize = input_raster
        # Extract to mask raster
        print('Extracting output to mask...')
        extract_raster = ExtractByMask(input_raster, mask_raster)
        # Set the snap raster and cell size environments to match the mask raster
        arcpy.env.snapRaster = mask_raster
        arcpy.env.cellSize = mask_raster
        # Project the extracted raster to NAD 1983 Alaska Albers using bilinear interpolation
        print('Reprojecting and resampling image tile to match area of interest...')
        projection = arcpy.SpatialReference(3338)
        arcpy.ProjectRaster_management(extract_raster, output_raster, projection, 'BILINEAR', '10 10', 'WGS_1984_(ITRF00)_To_NAD_1983')
        # End timing function execution and calculate elapsed time
        end = time.time()
        elapsed = int(end - start)
        success_time = datetime.datetime.now()
        # Report process success
        print('Successfully projected image tile...')
        print('----------')
        out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                    datetime.timedelta(seconds=elapsed))
        return out_process

In [12]:
# Run the projection process for each tile in list
for tile in tile_list:
    # Define output raster
    output_raster = os.path.join(tiles_project, os.path.split(tile)[1])
    # Define input and output arrays
    project_tile_inputs = [mask_raster, tile]
    project_tile_outputs = [output_raster]
    # Create key word arguments
    project_tile_kwargs = {'input_array' : project_tile_inputs,
                           'output_array' : project_tile_outputs
                          }
    # Project tile
    arcpy_geoprocessing(project_tile, check_output = False, **project_tile_kwargs)

----------
Processing raster tile Sent2_05May_11_shortInfrared1-0000000000-0000000000.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 17:31 (Elapsed time: 0:01:52)
----------
Processing raster tile Sent2_05May_11_shortInfrared1-0000000000-0000032768.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 17:31 (Elapsed time: 0:00:28)
----------
Processing raster tile Sent2_05May_12_shortInfrared2-0000000000-0000000000.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 17:33 (Elapsed time: 0:01:52)
----------
Processing raster tile Sent2_05May_12_shortInfrared2-0000000000-0000032768.ti

Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 18:14 (Elapsed time: 0:00:53)
----------
Processing raster tile Sent2_05May_ndwi-0000000000-0000032768.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 18:17 (Elapsed time: 0:03:47)
----------
Processing raster tile Sent2_05May_ndwi-0000000000-0000065536.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 18:18 (Elapsed time: 0:00:54)
----------
Processing raster tile Sent2_06June_11_shortInfrared1-0000000000-0000000000.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
--------

Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 18:59 (Elapsed time: 0:02:00)
----------
Processing raster tile Sent2_06June_ndsi-0000000000-0000032768.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 18:59 (Elapsed time: 0:00:31)
----------
Processing raster tile Sent2_06June_ndvi-0000000000-0000032768.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 19:03 (Elapsed time: 0:03:55)
----------
Processing raster tile Sent2_06June_ndvi-0000000000-0000065536.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeede

Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 19:45 (Elapsed time: 0:00:31)
----------
Processing raster tile Sent2_07July_ndmi-0000000000-0000000000.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 19:47 (Elapsed time: 0:02:01)
----------
Processing raster tile Sent2_07July_ndmi-0000000000-0000032768.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 19:48 (Elapsed time: 0:00:31)
----------
Processing raster tile Sent2_07July_ndsi-0000000000-0000000000.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeede

Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 20:32 (Elapsed time: 0:03:44)
----------
Processing raster tile Sent2_08August_8_nearInfrared-0000000000-0000065536.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 20:33 (Elapsed time: 0:00:51)
----------
Processing raster tile Sent2_08August_nbr-0000000000-0000000000.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 20:35 (Elapsed time: 0:01:58)
----------
Processing raster tile Sent2_08August_nbr-0000000000-0000032768.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
-----

Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 21:17 (Elapsed time: 0:00:29)
----------
Processing raster tile Sent2_09September_8a_redEdge4-0000000000-0000000000.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 21:19 (Elapsed time: 0:01:53)
----------
Processing raster tile Sent2_09September_8a_redEdge4-0000000000-0000032768.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 21:19 (Elapsed time: 0:00:29)
----------
Processing raster tile Sent2_09September_8_nearInfrared-0000000000-0000032768.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully proj

Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 22:04 (Elapsed time: 0:01:53)
----------
Processing raster tile Sent2_10October_6_redEdge2-0000000000-0000032768.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 22:05 (Elapsed time: 0:00:29)
----------
Processing raster tile Sent2_10October_7_redEdge3-0000000000-0000000000.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image tile...
----------
Succeeded at 2019-04-06 22:07 (Elapsed time: 0:01:53)
----------
Processing raster tile Sent2_10October_7_redEdge3-0000000000-0000032768.tif...
Extracting output to area of interest...
Reprojecting and resampling image tile to match area of interest...
Successfully projected image 